In [87]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
!pip install openai==0.28.0
!pip install huggingface_hub
!pip install groq
!pip install requests

In [89]:
import os
import re
from groq import Groq
from huggingface_hub import InferenceClient
import pandas as pd
import json
from google.colab import userdata

In [90]:
file_path = '/content/drive/My Drive/data.csv'
data = pd.read_csv(file_path)
case_row = data.iloc[497]
full_text = case_row['text']
case_summary = full_text
accusation = full_text
testimony_prompt = full_text

In [91]:
import os
import google.colab
from huggingface_hub import InferenceClient, login, HfFolder
from transformers import AutoTokenizer


hf_token = os.getenv("HF_TOKEN")


if hf_token and not HfFolder.get_token():
    login(token=hf_token)


hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B", token=hf_token) #tokenizer

#truncate the prompt
def truncate_prompt(prompt, max_input_tokens=3900):
    input_ids = hf_tokenizer(prompt, return_tensors="pt").input_ids[0]
    if len(input_ids) > max_input_tokens:
        prompt = hf_tokenizer.decode(input_ids[:max_input_tokens], skip_special_tokens=True)
    return prompt


In [92]:
import os
import time
import json
import regex
import requests

class Agent:
    class GroqHandler:
        def __init__(self):
            self.api_key = "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"
            self.base_url = "https://api.groq.com/openai/v1/chat/completions"
        def query(self, prompt, role_context):
            headers = {
                'Authorization': f'Bearer {self.api_key}',
                'Content-Type': 'application/json'
            }
            payload = {
                "model": "llama3-70b-8192",
                "messages": [
                    {"role": "system", "content": role_context},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": 300,
                "temperature": 0.3
            }
            rate_limited_once = False
            for attempt in range(3):
                try:
                    response = requests.post(
                        self.base_url,
                        headers=headers,
                        json=payload,
                        timeout=30
                    )
                    if response.status_code == 429:
                        wait_time = float(response.headers.get("Retry-After", 60))
                        if not rate_limited_once:
                            print(f"Rate limited. Waiting {wait_time}s (suppressed further messages)...")
                            rate_limited_once = True
                        time.sleep(wait_time)
                        continue
                    response.raise_for_status()
                    return response.json()['choices'][0]['message']['content'].strip()
                except Exception as e:
                    print(f"API Error: {str(e)}")
                    time.sleep(2 ** attempt)
            return "API request failed after retries"
    def __init__(self, name, role, llm_type):
        self.name = name
        self.role = role
        self.llm_type = llm_type.lower()
        self.memory = []
        if self.llm_type == "groq":
            self.client = Agent.GroqHandler()
        else:
            from huggingface_hub import InferenceClient
            self.client = InferenceClient("google/flan-t5-base")
    def generate_response(self, prompt):
        context = "\n".join([f"User: {m['prompt']}\n{self.name}: {m['response']}"
                           for m in self.memory[-2:]])
        full_prompt = f"{context}\nUser: {prompt}\n{self.name}:"
        try:
            if self.llm_type == "hf":
                response = self.client.text_generation(full_prompt, max_new_tokens=300)
            else:
                response = self.client.query(full_prompt, f"You are {self.name}, a {self.role}")
            self.memory.append({"prompt": prompt, "response": response})
            return response
        except Exception as e:
            print(f"Response error: {str(e)}")
            return "Error generating response"

class JudgeDrivenTrial:
    def __init__(self, case_summary):
        self.case_summary = case_summary[:500]
        self.agents = {}
        self.phase_sequence = [
            "opening",
            "direct-examination",
            "cross-examination",
            "closing",
            "jury-deliberation"
        ]
        self.current_phase_index = 0
        self.examination_record = []
        self._init_core_agents()
    def _init_core_agents(self):
        self.add_agent("Justice Ray", "judge", "groq")
        self.add_agent("Ava", "prosecutor", "groq")
        self.add_agent("Leo", "defense", "groq")
        self.add_agent("Emily", "plaintiff", "groq")
        self.add_agent("Jury Foreman", "jury", "groq")
        self.judge = self.agents["judge"]
    def add_agent(self, name, role, llm_type):
        if role.lower() in self.agents:
            raise ValueError(f"Role {role} already exists")
        self.agents[role.lower()] = Agent(name, role, llm_type)
    def _safe_json_parse(self, raw_response):
        try:
            clean = regex.sub(r'``````', '', raw_response, flags=regex.DOTALL)
            match = regex.search(r'\{(?:[^{}]|(?R))*\}', clean, regex.DOTALL)
            return json.loads(match.group()) if match else {}
        except Exception as e:
            print(f"JSON Parse Error: {str(e)}")
            return {}
    def judicial_order(self, prompt):
        response = self.judge.generate_response(f"{prompt} Respond with valid JSON only.")
        return self._safe_json_parse(response)
    def conduct_trial(self):
        self._open_court()
        while self.current_phase_index < len(self.phase_sequence):
            current_phase = self.phase_sequence[self.current_phase_index]
            print(f"\n=== {current_phase.replace('-', ' ').title()} Phase ===")
            if current_phase == "opening":
                self._handle_openings()
            elif current_phase == "direct-examination":
                self._handle_direct_examination()
            elif current_phase == "cross-examination":
                self._handle_cross_examination()
            elif current_phase == "closing":
                self._handle_closing()
            elif current_phase == "jury-deliberation":
                self._handle_jury_deliberation()
            self.current_phase_index += 1
        self._deliver_verdict()
    def _open_court(self):
        print(f"\nJudge {self.judge.name}: Court is now in session.")
        print(f"Case: {self.case_summary}...")
    def _handle_openings(self):
        order = self.judicial_order('''{"order": ["plaintiff", "prosecutor", "defense"]}''') \
                .get("order", ["plaintiff", "prosecutor", "defense"])
        for role in order:
            agent = self.agents.get(role.lower())
            if agent:
                statement = agent.generate_response(f"Opening statement: {self.case_summary}")
                print(f"\n{agent.role.title()}: {statement[:300]}...")
                time.sleep(1)
    def _handle_direct_examination(self):
        if witness := self.agents.get("witness"):
            print(f"\nDirect examination of witness: {witness.name}")
            self.examination_record = []
            for q_num in range(2):
                question = self.agents["prosecutor"].generate_response(
                    f"Direct question {q_num+1} for {witness.name} about {self.case_summary[:200]}"
                )
                response = witness.generate_response(question)
                self.examination_record.append((question, response))
                print(f"\nQ{q_num+1}: {question[:150]}\nA: {response[:200]}...")
                time.sleep(1)
    def _handle_cross_examination(self):
        if witness := self.agents.get("witness"):
            print(f"\nCross-examination of witness: {witness.name}")
            for i, (question, response) in enumerate(self.examination_record):
                cross_q = self.agents["defense"].generate_response(
                    f"Cross-examine this testimony: Q: {question} A: {response}"
                )
                cross_a = witness.generate_response(cross_q)
                print(f"\nDefense Cross-Q{i+1}: {cross_q[:150]}\nA: {cross_a[:200]}...")
                # Optionally, let prosecutor rebut
                rebuttal = self.agents["prosecutor"].generate_response(
                    f"Rebut the defense's cross-examination: {cross_q} | Witness answer: {cross_a}"
                )
                print(f"\nProsecutor Rebuttal: {rebuttal[:200]}...")
                time.sleep(1)
    def _handle_closing(self):
        print("\nClosing Arguments:")
        for role in ["plaintiff", "prosecutor", "defense"]:
            agent = self.agents.get(role)
            if agent:
                closing = agent.generate_response("Summarize key points")
                print(f"\n{agent.role.title()}: {closing[:300]}...")
                time.sleep(1)
    def _handle_jury_deliberation(self):
        print("\nJury Deliberation:")
        if jury := self.agents.get("jury"):
            deliberation = jury.generate_response(
                "Discuss the evidence and arguments presented in the case and recommend a verdict"
            )
            print(f"\nJury Foreman: {deliberation[:300]}...")
            time.sleep(1)
    def _deliver_verdict(self):
        jury_recommendation = self.agents["jury"].generate_response(
            "Based on all evidence and arguments, recommend a verdict (guilty/not guilty/remanded)"
        )
        verdict_data = self.judicial_order(f'''{{
            "verdict": "{jury_recommendation}",
            "reasoning": {{
                "jurisdiction": "<analysis>",
                "procedure": "<evaluation>",
                "legal_basis": "<citations>"
            }}
        }}''')
        print("\n=== FINAL JUDGMENT ===")
        print(f"Jury Recommendation: {jury_recommendation[:200]}")
        print(f"Court Verdict: {verdict_data.get('verdict', 'Case dismissed')}")
        print("Legal Reasoning:")
        for k, v in verdict_data.get("reasoning", {}).items():
            print(f"- {k.title()}: {v[:200] or 'Not addressed'}...")
        print("\nCourt is adjourned.")

# ---- Usage ----
if __name__ == "__main__":
    trial = JudgeDrivenTrial(case_summary)
    trial.add_agent("Bob", "witness", "groq")
    trial.conduct_trial()




Judge Justice Ray: Court is now in session.
Case: CIVIL APPELLATE JURISDICTION Civil Appeal No. 220 of 1964. Appeal from the judgment and order dated April 17, 1961 of the Calcutta High Court in Appeal from Original Order No. 11 5 of 1960. Chowdhury, S. Mukherjee and S. N. Mukherjee, for the appellant. Sen, V. A. Seyid Muhammad, P. K. Das and P. K. Bose for the respondents. The Judgment of the Court was delivered by Gajendragadkar, C.J. This appeal arises out of a suit filed by the appellant, Mirza Ali Akbar Kashani, against the two respondents...

=== Opening Phase ===

Plaintiff: Your Honor, esteemed members of the court, I, Emily, am the plaintiff in this case, and I am represented by my counsel, Sen, V. A. Seyid Muhammad, P. K. Das, and P. K. Bose. I am here today to seek justice and redress for the grievances that I have suffered at the hands of the appellant, Mirza Ali A...

Prosecutor: Honorable Chief Justice Gajendragadkar and esteemed members of the court, I, Ava, prosecutor 